In [1]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(HDtweedie)
library(TDboost)
library(mgcv)
library(MASS)
library(glmnet)

Loading required package: coda

Loading required package: Matrix

Loading required package: splines

Loading required package: lattice

Loaded TDboost 1.2


Loading required package: nlme


Attaching package: 'nlme'


The following objects are masked from 'package:cplm':

    fixef, ranef, VarCorr


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.

Loaded glmnet 4.1-2



In [93]:
source("S_U.R")

In [78]:
x1<-runif(10000,2,8)
x2<-runif(10000,-10,1)
x3<-runif(10000,-2,0)
b_t<-runif(4,-2,1)
design_x <- as.matrix(cbind(x1,x2,x3,x1*x2))
colnames(design_x) <- c('x1','x2','x3','x1:x2')
mu<-exp(design_x%*%b_t)
summary(mu)

       V1           
 Min.   :  0.00000  
 1st Qu.:  0.00257  
 Median :  0.04557  
 Mean   :  2.33736  
 3rd Qu.:  0.58361  
 Max.   :123.33127  

In [80]:
y_sim<- rTweedie(mu,p=1.5,phi=10)
length(which(y_sim==0))

[1] 8878

In [81]:
dd<-as.data.frame(cbind(design_x[,1:3],y_sim))

In [82]:
sam <- sample(rep(1:2,len=10000))
sim_train<-dd[sam==1,]
sim_test<-dd[sam==2,]
sim_train<-as.data.frame(sim_train)
sim_test<-as.data.frame(sim_test)
colnames(sim_train)[4]='y'
colnames(sim_test)[4]='y'

In [83]:
#Initial: For example, we can use the least square estimates or the simple regression estimates by regressing the response y on each of the terms.
# Main effects
fit_b1<-glm(y~x1-1,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
b1_0 <- fit_b1$coefficients

fit_b2<-glm(y~x2-1,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
b2_0 <- fit_b2$coefficients

fit_b3<-glm(y~x3-1,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
b3_0 <- fit_b3$coefficients

#Interaction: when update r12, use y~b1b2x1x2 with no intercept.
effect_12<-b1_0*b2_0*sim_train$x1*sim_train$x2
fit_r12<-glm(sim_train$y~effect_12-1,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
r12_0<-fit_r12$coefficients

effect_13<-b1_0*b3_0*sim_train$x1*sim_train$x3
fit_r13<-glm(sim_train$y~effect_13-1,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
r13_0<-fit_r13$coefficients

effect_23<-b1_0*b2_0*sim_train$x2*sim_train$x3
fit_r23<-glm(sim_train$y~effect_23-1,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
r23_0<-fit_r23$coefficients

In [84]:
sx<-model.matrix(~.^2,data=sim_train[,1:3])

In [85]:
base<-glmnet(x=sx, y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),intercept=FALSE,control=list(maxit=500))
lambb<-base$lambda

In [86]:
beta_0_t<-c(b1_0, b2_0, b3_0)
r_0_t<-c(r12_0,r13_0,r23_0)

In [112]:
coef_result<-shim_lasso_update(sim_train[,1:3],sim_train$y,1.5,lamb_r=lambb[1],lamb_b=lambb[1],r_0_t,beta_0_t)

ERROR: Error in while (initial_per >= 0.005) {: missing value where TRUE/FALSE needed


In [102]:
r_0_t
beta_0_t

effect_12  effect_13  effect_23 
  2.975899   1.567374 -15.686175

x1         x2         x3 
 0.0487587 -0.1762281 -0.5831050

In [113]:
x=sim_train[,1:3]
y=sim_train$y
power=1.5
lamb_r=lambb[1]
lamb_b=lambb[1]

In [132]:
initial_per<-999
tmp_r<-r_0_t
tmp_b<-beta_0_t
n<-length(tmp_b)
index<-combn(c(1:n),m=2)
n_c=0

In [136]:
index<-combn(c(1:n),m=2)    
total_effect=0
for (i in 1:n){
    tmp_effect<-tmp_r[i]*tmp_b[index[1,i]]*tmp_b[index[2,i]]*x[,index[1,i]]*x[,index[2,i]]
    total_effect=total_effect+tmp_effect
}
tmp_fitted<-exp(as.matrix(x)%*%tmp_b+total_effect)
Q_1<-Q(y,tmp_fitted,power,lamb_r,lamb_b,tmp_r,tmp_b)
    

In [150]:
summary(log(dtweedie(y=y,mu=tmp_fitted,phi=mean(tweedie.dev(y=y,mu=tmp_fitted,power=power)),power=power)))

      Min.    1st Qu.     Median       Mean    3rd Qu.       Max. 
      -Inf -7.469e-06 -2.710e-06       -Inf -1.738e-07 -8.000e-12 

In [116]:
design_matrix<-c()
    for (i in 1:n){
        beta_tmp<-tmp_b[index[1,i]]*tmp_b[index[2,i]]
        design_matrix<-cbind(design_matrix,beta_tmp*x[,index[1,i]]*x[,index[2,i]]  )
    }

In [117]:
all(design_matrix==0)

[1] FALSE

In [118]:
tmp_r<-r_update(tmp_b,power=power,y=y,X=x,lambda=lamb_r)
        tmp_b<-beta_update(tmp_b,tmp_r,power=power,y=y,X=x,lambda=lamb_b) 
        total_effect=0
        for (i in 1:n){
            tmp_effect<-tmp_r[i]*tmp_b[index[1,i]]*tmp_b[index[2,i]]*x[,index[1,i]]*x[,index[2,i]]
            total_effect=total_effect+tmp_effect
        }
        tmp_fitted<-exp(as.matrix(x)%*%tmp_b+total_effect)
        Q_2<-Q(y,tmp_fitted,power,lamb_r,lamb_b,tmp_r,tmp_b)
        initial_per<-abs(Q_1-Q_2)/abs(Q_1)

In [122]:
Q_2
Q_1

[1] 4855.691

[1] Inf

In [120]:
 initial_per

[1] NaN

In [119]:
initial_per>=0.05

[1] NA

In [ ]:
design_matrix<-c()
    for (i in 1:n){
        beta_tmp<-tmp_b[index[1,i]]*tmp_b[index[2,i]]
        design_matrix<-cbind(design_matrix,beta_tmp*x[,index[1,i]]*x[,index[2,i]]  )
    }
    if (all(design_matrix==0)){
        result=data.frame('beta'=tmp_b,'r'=tmp_r,'n'=rep(n_c,length(tmp_b)))
        return(result)
        break
    }
        tmp_r<-r_update(tmp_b,power=power,y=y,X=x,lambda=lamb_r)
        tmp_b<-beta_update(tmp_b,tmp_r,power=power,y=y,X=x,lambda=lamb_b) 
        total_effect=0
        for (i in 1:n){
            tmp_effect<-tmp_r[i]*tmp_b[index[1,i]]*tmp_b[index[2,i]]*x[,index[1,i]]*x[,index[2,i]]
            total_effect=total_effect+tmp_effect
        }
        tmp_fitted<-exp(as.matrix(x)%*%tmp_b+total_effect)
        Q_2<-Q(y,tmp_fitted,power,lamb_r,lamb_b,tmp_r,tmp_b)
        initial_per<-abs(Q_1-Q_2)/abs(Q_1)
        n_c=n_c+1


In [36]:
fitted<-ts_fitted(sim_train[,1:3],coef_result[,1],coef_result[,2])

In [37]:
power=1.5
mean(tweedie.dev(sim_train$y,fitted,power))

[1] 2.255731

In [95]:
r_lamb<-lambb
beta_lamb<-lambb
n_lamb<-length(beta_lamb)
power=1.5
final_result=matrix(nrow=n_lamb,ncol=n_lamb)
for (i in 1:n_lamb){
    for (j in 1:n_lamb){
        coef_result<-shim_lasso_update(sim_train[,1:3],sim_train$y,power,lamb_r=r_lamb[i],lamb_b=beta_lamb[j],r_0_t,beta_0_t)
        fitted<-ts_fitted(sim_train[,1:3],coef_result[,1],coef_result[,2])
        final_result[i,j]=sum(log(dtweedie(sim_train$y,mu=fitted,phi=sum(tweedie.dev(sim_train$y,fitted,1.5))/length(sim_train$y),power=1.5)))
    }
}

ERROR: Error in while (initial_per >= 0.005) {: missing value where TRUE/FALSE needed


In [58]:
coef_result<-shim_lasso_update(sim_train[,1:3],sim_train$y,power,lamb_r=r_lamb[15],lamb_b=beta_lamb[10],r_0_t,beta_0_t)
fitted<-ts_fitted(sim_train[,1:3],coef_result[,1],coef_result[,2])

In [59]:
coef_result


,beta,r,n
,<dbl>,<dbl>,<dbl>
V1,0,0,1
V2,0,0,1
V3,0,0,1


In [55]:
coef_result<-shim_lasso_update(sim_train[,1:3],sim_train$y,power,lamb_r=r_lamb[5],lamb_b=beta_lamb[5],r_0_t,beta_0_t)
coef_result

,beta,r,n
,<dbl>,<dbl>,<dbl>
V1,0,0,1
V2,0,0,1
V3,0,0,1
